<a href="https://colab.research.google.com/github/masadeghi/EHRsample/blob/main/final/fmri_dementia_imaging_data_preprocessing_jdim_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import dependencies

In [ ]:
import os
import gc
import copy
from tqdm.notebook import tqdm
from itertools import product

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score

from joblib import Parallel, delayed

!pip install -qU tsfresh
import tsfresh
from tsfresh.feature_extraction import extract_features

!pip install -qU tsai
import tsai
from tsai.utils import to_tsfresh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.1/169.1 kB 9.4 MB/s eta 0:00:00


# Connect to Google Drive and set cd

In [ ]:
# Mount Google drive
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/Coding projects/resting_fmri_dementia/

/content/gdrive/MyDrive/Coding projects/resting_fmri_dementia


# Data preprocessing:

## Import data into usable form by Python

### Import labels

In [ ]:
labels = pd.read_csv('./group_csv_files/labels_new_final.csv')

In [ ]:
labels['class'].replace(to_replace = {'AD': 0, 'FTD': 1, 'HC': 2, 'MCI' : 3}, inplace = True)

labels.head()

,class,patient_original_id,fmri_id_in_database,subject_id_in_dataset,valid_scans,invalid_scans,perc_invalid
0,0,002_S_5018,rs_fMRI_1',1.txt,111,29,0.21
1,0,002_S_5018,rs_fMRI_2',2.txt,103,37,0.26
2,0,002_S_5018,rs_fMRI_3',3.txt,129,11,0.08
3,0,002_S_5018,rs_fMRI_4',4.txt,121,19,0.14
4,0,006_S_4153,rs_fMRI_1',5.txt,129,11,0.08


### Function for retrieving information from the .txt files into numpy arrays


In [ ]:
def get_datasets(data_csv, standardized=True):

  """
  Use a CSV file containing dataset metadata to create dataset paths. Then load and process these datasets.

  Parameters:
  ----------
  data_csv : pandas.DataFrame
      A DataFrame where each row contains metadata for each dataset including
      'subject_id_in_dataset' and 'class'.
  standardized : bool, optional
      A flag indicating whether to load standardized (scaled) data. If True, data
      is loaded from the standardized data directory ('./scaled_data_dir_final').
      Otherwise, data is loaded from the raw data directory ('./raw_data_dir_final').
      Default is True.

  Returns:
  -------
  X : numpy.ndarray
      A 3D NumPy array of shape (n_samples, 200, 140) containing the dataset matrices.
      Each matrix is truncated or padded to a shape of (200, 140).
  y : numpy.ndarray
      A 1D NumPy array of shape (n_samples,) containing the class labels for each dataset.

  Notes:
  -----
  - The function assumes that the data matrices are stored in CSV format and can be read
    using `numpy.genfromtxt` with a comma delimiter.
  - If a matrix has fewer than 200 rows or 140 columns, the corresponding entries in `X`
    will remain as NaNs.
  - Progress of the dataset loading process is displayed using `tqdm`.

  Example:
  -------
  data_csv = pd.DataFrame({
      'subject_id_in_dataset': ['1.txt', '2.txt'],
      'class': [0, 1]
  })
  X, y = get_datasets(data_csv)
  """

  X = np.empty((len(data_csv), 200, 140))
  X[:] = np.nan

  y = np.empty(len(data_csv))
  y[:] = np.nan

  for i, row in tqdm(data_csv.iterrows(), total=data_csv.shape[0]):
    matrix_name = row['subject_id_in_dataset']
    matrix_label = row['class']

    if standardized:
      dir_path = './scaled_data_dir_final'
    else:
      dir_path = './raw_data_dir_final'

    matrix_path = os.path.join(dir_path, matrix_name)
    matrix_arr = np.genfromtxt(matrix_path, delimiter=',')

    if matrix_arr.shape[0] >= 200 and matrix_arr.shape[1] >= 140:
      matrix_arr = matrix_arr[:200, :140]
      X[i] = matrix_arr
      y[i] = matrix_label

  return X, y

### Load .txt datasets as arrays

In [ ]:
# Load data from .txt files
X, y = get_datasets(labels, standardized = False)

print("X shape: ", X.shape)
print("y shape: ", y.shape)

100%|██████████| 1351/1351 [06:15<00:00,  3.60it/s]

X shape:  (1351, 200, 140)
y shape:  (1351,)


## Clean up the datasets

1. Remove elements that had a high percentage of outlier scans (based on labels dataframe)

2. Remove the fMRIs that had returned all zeros (dimensions smaller than 200 * 140) during preprocessing in CONN. We do this by removing any of the samples that have NaNs.

In [ ]:
# Indexes of subjects with <= 50% outlier scans
out_idx = np.where(~(labels['perc_invalid'] > 0.5))[0]

# Indexes of subjects without NaNs
nan_idx = np.unique(np.where(~np.isnan(X).any(axis=2))[0])

# Indexes of subjects who satisfy both criteria
final_idx = np.array(list(set(out_idx).intersection(nan_idx)))

# Indexes of subjects that were excluded
excluded_idx = np.array(set(list(labels.index)).difference(final_idx))

In [ ]:
# Save indexes
np.save('./group_csv_files/final_idx.npy', final_idx)
np.save('./group_csv_files/excluded_idx.npy', excluded_idx)

In [ ]:
# Number of scans in each class
labels['class'].value_counts().sort_index()

0    101
1    396
2    470
3    384
Name: class, dtype: int64

In [ ]:
# Number of patients in each class, seggregated by database:
grouped_by_class_adni_ids = labels[labels['patient_original_id'].str.len() == 10].groupby('class').patient_original_id.unique()
grouped_by_class_nifd_ids = labels[labels['patient_original_id'].str.len() == 8].groupby('class').patient_original_id.unique()

print("Patients in AD class from ADNI: ", len(grouped_by_class_adni_ids[0]))
print("Patients in FTD class from NIFD: ", len(grouped_by_class_nifd_ids[1]))
print("Patients in HC class from ADNI, NIFD, and total: ",
      len(grouped_by_class_adni_ids[2]),
      len(grouped_by_class_nifd_ids[2]),
      len(grouped_by_class_adni_ids[2]) + len(grouped_by_class_nifd_ids[2]))
print("Patients in MCI class from ADNI: ", len(grouped_by_class_adni_ids[3]))

Patients in AD class from ADNI:  33
Patients in FTD class from NIFD:  171
Patients in HC class from ADNI, NIFD, and total:  51 114 165
Patients in MCI class from ADNI:  105


In [ ]:
# Number of patients excluded from each class
labels.loc[excluded_idx, 'class'].value_counts().sort_index()

0     11
1    133
2     86
3     37
Name: class, dtype: int64

In [ ]:
# Number of patients in each class after the exclusions
labels.loc[final_idx, 'class'].value_counts().sort_index()

0     90
1    263
2    384
3    347
Name: class, dtype: int64

In [ ]:
# Get new X and y without the exlusions
X = X[final_idx, :, :]
y = y[final_idx]

print("X shape: ", X.shape)
print("y shape: ", y.shape)

X shape:  (1084, 200, 140)
y shape:  (1084,)


In [ ]:
# Get the new labels subset without the exclusions
labels = labels.loc[final_idx, :]

In [ ]:
# Number of patients in each class, seggregated by database, after the exclusions:
grouped_by_class_adni_ids = labels[labels['patient_original_id'].str.len() == 10].groupby('class').patient_original_id.unique()
grouped_by_class_nifd_ids = labels[labels['patient_original_id'].str.len() == 8].groupby('class').patient_original_id.unique()

print("Patients in AD class from ADNI: ", len(grouped_by_class_adni_ids[0]))
print("Patients in FTD class from NIFD: ", len(grouped_by_class_nifd_ids[1]))
print("Patients in HC class from ADNI, NIFD, and total: ",
      len(grouped_by_class_adni_ids[2]),
      len(grouped_by_class_nifd_ids[2]),
      len(grouped_by_class_adni_ids[2]) + len(grouped_by_class_nifd_ids[2]))
print("Patients in MCI class from ADNI: ", len(grouped_by_class_adni_ids[3]))

Patients in AD class from ADNI:  32
Patients in FTD class from NIFD:  151
Patients in HC class from ADNI, NIFD, and total:  51 97 148
Patients in MCI class from ADNI:  103


## Train/val/test split
Performed based on patient_IDs and their diagnosis at their first visit, rather than image_IDs to prevent data leakage. Some patients have multiple associated image_IDs

In [ ]:
splittable_data = labels.loc[:, ['class', 'patient_original_id']].groupby('patient_original_id').first()

In [ ]:
# Splitting out the test set
ids_train_val, ids_test = train_test_split(splittable_data, test_size = 0.2, shuffle = True,
                                           random_state = 42, stratify = splittable_data['class'])

In [ ]:
train_val_idx = np.where(labels['patient_original_id'].isin(ids_train_val.index).values)[0]
test_idx = np.where(labels['patient_original_id'].isin(ids_test.index).values)[0]

In [ ]:
# Shuffle the indexes
np.random.seed(42)
np.random.shuffle(train_val_idx)

np.random.seed(42)
np.random.shuffle(test_idx)

In [ ]:
X_train_val = X[train_val_idx, :, :]
y_train_val = y[train_val_idx]

X_test = X[test_idx, :, :]
y_test = y[test_idx]

## Save raw datasets and indexes


In [ ]:
np.save('datasets_as_np_arrays/X_final_train_val_paper.npy', X_train_val)
np.save('datasets_as_np_arrays/y_final_train_val_paper.npy', y_train_val)

np.save('datasets_as_np_arrays/X_final_test.npy', X_test)
np.save('datasets_as_np_arrays/y_final_test.npy', y_test)

# Save index lists to disk
np.save('datasets_as_np_arrays/train_val_idx_paper.npy', train_val_idx)
np.save('datasets_as_np_arrays/test_idx_paper.npy', test_idx)

## Standardize the data

1. Get the means and standard deviations across all of the training set fMRIs
2. Standardize both the training and test sets using the calculated means and standard deviations
3. Save the scaled datasets

In [ ]:
train_mean = np.mean(X_train_val)
train_std = np.std(X_train_val, ddof=0)
train_mean, train_std

(-7.847354719947522e-05, 3.312373340764462)

In [ ]:
X_scaled = (X - train_mean)/train_std
X_train_val_scaled = (X_train_val - train_mean)/train_std
X_test_scaled = (X_test - train_mean)/train_std

In [ ]:
# # Save to disk
np.save('scaled_datasets_as_np_arrays/X_final_train_val_paper.npy', X_train_val_scaled)
np.save('scaled_datasets_as_np_arrays/y_final_train_val_paper.npy', y_train_val)

np.save('scaled_datasets_as_np_arrays/X_final_test_paper.npy', X_test_scaled)
np.save('scaled_datasets_as_np_arrays/y_final_test_paper.npy', y_test)

# Feature extraction

The feature extraction steps are performed on ALL of the data because they don't leak information

## Import data in memmory map mode

In [ ]:
# Scaled_data
X = np.load('./data_dir_npy/X_imaging.npy', mmap_mode = 'c')
# y = np.load('scaled_datasets_as_np_arrays/y_final.npy', mmap_mode = 'c')

In [ ]:
X_df = to_tsfresh_df(X)

In [ ]:
X_df.shape

(151760, 201)

In [ ]:
X_df.id[:5]

0    0
1    0
2    0
3    0
4    0
Name: id, dtype: int64

## Determine the features to be extracted

More information on each feature may be viewed here: https://tsfresh.readthedocs.io/en/latest/text/list_of_features.html

In [ ]:
fc_parameters = {
    "absolute_sum_of_changes": None,
    "first_location_of_maximum": None,
    "first_location_of_minimum": None,
    "longest_strike_above_mean": None,
    "longest_strike_below_mean": None,
    "approximate_entropy": [
                    {"m": 2, "r": r} for r in [0.1, 0.3, 0.5, 0.7, 0.9] # default: [0.1, 0.3, 0.5, 0.7, 0.9]
                ], #high_comp_cost
    "c3": [{"lag": lag} for lag in range(1, 4)], # default: range(1, 4)
    "cid_ce": [{"normalize": True}, {"normalize": False}],
    "cwt_coefficients": [
                    {"widths": width, "coeff": coeff, "w": w}
                    for width in [(2, 5, 10, 20)] # default: [(2, 5, 10, 20)]
                    for coeff in range(15) # default: range(15)
                    for w in (2, 5, 10, 20) # default: (2, 5, 10, 20)
                ],
    "fft_coefficient": [
                    {"coeff": k, "attr": a}
                    for a, k in product(["real", "imag", "abs", "angle"], range(100)) # default: product(["real", "imag", "abs", "angle"], range(100)
                ],
    "lempel_ziv_complexity": [{"bins": x} for x in [2, 3, 5, 10, 100]], # default: [2, 3, 5, 10, 100]
    "ratio_beyond_r_sigma":  [
                    {"r": x} for x in [0.5, 1, 1.5, 2, 2.5, 3, 5, 6, 7, 10] # default: [0.5, 1, 1.5, 2, 2.5, 3, 5, 6, 7, 10]
                ],
    "spkt_welch_density": [{"coeff": coeff} for coeff in [2, 5, 8]]
}

## Perform the feature extraction

In [ ]:
# Prepares the time series array to be used as a tsfresh dataset to allow feature extraction

X_df = to_tsfresh_df(X)

if y.ndim == 1:
  y = y.reshape(-1,1)

In [ ]:
features_df_custom = extract_features(X_df, column_id = "id", n_jobs = 2, default_fc_parameters = fc_parameters)

Feature Extraction:  60%|██████    | 6/10 [31:11<17:10, 257.54s/it]

In [ ]:
features_df_custom['target'] = y
features_df_custom.to_csv('./features_data/features_final_df_custom_manuscript.csv', index = False)

## Clean the features data

In [ ]:
# Drop columns with NaNs
features_df_custom_mod = features_df_custom.iloc[:, :-1].dropna(axis = 1)

# Standardize the features
features_df_custom_mod_train = features_df_custom_mod.iloc[train_idx, :]
scaler = StandardScaler()
scaler.fit(features_df_custom_mod_train)
features_df_custom_mod_scaled = scaler.transform(features_df_custom_mod)

# Save the cleaned and scaled version
features_df_custom_mod_scaled_df = pd.DataFrame(features_df_custom_mod_scaled)
features_df_custom_mod_scaled_df.columns = features_df_custom_mod.columns
features_df_custom_mod_scaled_df['target'] = y
features_df_custom_mod_scaled_df.to_csv('./features_data/features_final_df_custom_mod.csv', index = False)

## Does custom features data contain useful information regarding classification?

In [ ]:
features_df = pd.read_csv('./features_data/features_final_df_custom_mod.csv')

train_val_idx = np.load('datasets_as_np_arrays/train_val_idx.npy')
train_idx = np.load('datasets_as_np_arrays/train_idx.npy')
val_idx = np.load('datasets_as_np_arrays/val_idx.npy')
test_idx = np.load('datasets_as_np_arrays/test_idx.npy')

In [ ]:
y = features_df['target']
X = features_df.drop('target', axis = 1)

X_train, y_train = X.loc[train_idx, :], y[train_idx]
X_val, y_val = X.loc[val_idx, :], y[val_idx]
X_train_val, y_train_val = X.loc[train_val_idx, :], y[train_val_idx]
X_test, y_test = X.loc[test_idx, :], y[test_idx]

In [ ]:
rfc = RandomForestClassifier(random_state = 42)

rfc.fit(X_train, y_train)
train_pred = rfc.predict(X_train)
val_pred = rfc.predict(X_val)
test_pred = rfc.predict(X_test)

print("Balanced accuracy in train set:", balanced_accuracy_score(y_train, train_pred))
print("Balanced accuracy in validation set:", balanced_accuracy_score(y_val, val_pred))
print("Balanced accuracy in test set:", balanced_accuracy_score(y_test, test_pred))

Balanced accuracy in train set: 1.0
Balanced accuracy in validation set: 0.7916666666666667
Balanced accuracy in test set: 0.75
